In [1]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

In [2]:
h2o.init()

nvidia_df = h2o.import_file("../data/nvidia_stock_features_selected.csv")
microsoft_df = h2o.import_file("../data/microsoft_stock_features_selected.csv")
apple_df = h2o.import_file("../data/apple_stock_features_selected.csv")

target = 'Close'

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 23.0.2+7-58, mixed mode, sharing)
  Starting server from C:\Users\athar\anaconda3\envs\ds_autoML\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\athar\AppData\Local\Temp\tmp60g4441t
  JVM stdout: C:\Users\athar\AppData\Local\Temp\tmp60g4441t\h2o_athar_started_from_python.out
  JVM stderr: C:\Users\athar\AppData\Local\Temp\tmp60g4441t\h2o_athar_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,4 months and 6 days
H2O_cluster_name:,H2O_from_python_athar_azl03h
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.941 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [3]:
def train_h2o_automl(stock_name, df):
    print(f"\nTraining H2O AutoML for {stock_name}.")

    # Split train/test sets (80/20 split)
    train_size = int(0.8 * df.shape[0])
    train_data = df[:train_size, :]
    test_data = df[train_size:, :]

    # Convert target column to float
    train_data[target] = train_data[target].asnumeric()
    test_data[target] = test_data[target].asnumeric()

    # Train AutoML
    aml = H2OAutoML(max_models=20, seed=42, stopping_metric="MSE")
    aml.train(y=target, training_frame=train_data)

    # Get best model
    best_model = aml.leader

    # Make predictions
    predictions = best_model.predict(test_data).as_data_frame()

    # Evaluate performance
    from sklearn.metrics import mean_squared_error, r2_score
    actuals = test_data[target].as_data_frame().values.flatten()
    mse = mean_squared_error(actuals, predictions)
    r2 = r2_score(actuals, predictions)

    print(f"H2O AutoML Results for {stock_name}:")
    print(f"MSE: {mse:.6f}")
    print(f"R2 Score: {r2:.6f}")

    return best_model, mse, r2


In [4]:
nvidia_h2o_model, nvidia_h2o_mse, nvidia_h2o_r2 = train_h2o_automl("NVIDIA", nvidia_df)


Training H2O AutoML for NVIDIA.
AutoML progress: |
16:40:22.274: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


c:\Users\athar\anaconda3\envs\ds_autoML\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


H2O AutoML Results for NVIDIA:
MSE: 0.753727
R2 Score: 0.999538


c:\Users\athar\anaconda3\envs\ds_autoML\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [5]:
microsoft_h2o_model, microsoft_h2o_mse, microsoft_h2o_r2 = train_h2o_automl("Microsoft", microsoft_df)


Training H2O AutoML for Microsoft.
AutoML progress: |
16:57:50.117: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
H2O AutoML Results for Microsoft:
MSE: 19644.358645
R2 Score: -0.529286


c:\Users\athar\anaconda3\envs\ds_autoML\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
c:\Users\athar\anaconda3\envs\ds_autoML\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [6]:
apple_h2o_model, apple_h2o_mse, apple_h2o_r2 = train_h2o_automl("Apple", apple_df)


Training H2O AutoML for Apple.
AutoML progress: |
17:15:38.787: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
H2O AutoML Results for Apple:
MSE: 10346.705318
R2 Score: -1.418452


c:\Users\athar\anaconda3\envs\ds_autoML\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
c:\Users\athar\anaconda3\envs\ds_autoML\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
